# Import des modules

In [36]:
from eppy.modeleditor import IDF
import pandas as pd
from pathlib import Path

# Définition des fonctions utiles

In [37]:
def add_output_variables(idf):
    """ Cette fonction prend en entrée le chemin d'accès d'un fichier idf et en ressort le fichier idf auquel on a ajouté la même liste de données de sortie"""
    
    idf.idfobjects['OUTPUT:VARIABLE'] = []
        
    lis_out = ['Zone Mean Air Temperature',
            'Zone Mean Radiant Temperature',
            'Zone Operative Temperature',
            'Zone Ventilation Air Change Rate',
            'Zone Air Relative Humidity',
            'Zone Thermal Comfort Mean Radiant Temperature', # Thermal Comfort
            'Zone Thermal Comfort Operative Temperature', # Thermal Comfort
            'Zone Thermal Comfort Fanger Model PMV', # Thermal Comfort
            'Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature', # Thermal Comfort
            'Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature', # Thermal Comfort
            'Zone Thermal Comfort CEN 15251 Adaptive Model Temperature', # Thermal Comfort
            'Zone Thermal Comfort CEN 15251 Adaptive Model Running Average Outdoor Air Temperature', # Thermal Comfort
            'Zone Thermal Comfort Pierce Model Standard Effective Temperature', # Thermal Comfort
            'Site Outdoor Air Drybulb Temperature',
            'Site Direct Solar Radiation Rate per Area',
            'Site Diffuse Solar Radiation Rate per Area',
            'Site Wind Speed',
            'Site Sky Temperature',
            'Site Solar Azimuth Angle',
            'Site Solar Altitude Angle',
            'Site Solar Hour Angle',
            'Surface Outside Face Convection Heat Transfer Coefficient',
            'Surface Inside Face Convection Heat Transfer Coefficient',
            'Surface Outside Face Thermal Radiation to Air Heat Transfer Coefficient',
            'Surface Outside Face Thermal Radiation to Sky Heat Transfer Coefficient',
            'Surface Outside Face Thermal Radiation to Ground Heat Transfer Coefficient',
            'Surface Outside Face Temperature',
            'Surface Inside Face Temperature',
            'Surface Outside Face Incident Solar Radiation Rate per Area']


    lis_zone = ['RDC:RUE',
                'RDC:COUR',
                'RDC:CIRCULATION',
                'ETAGE:RUE',
                'ETAGE:COUR',
                'ETAGE:CIRCULATION',
                'COMBLES:RUE',
                'COMBLES:COUR',
                'COMBLES:CICULATION']
            
    for i in range(len(lis_out)):
        variable_name = lis_out[i]
        if variable_name.startswith('Site'):
            out = idf.newidfobject('OUTPUT:VARIABLE')
            out.Key_Value = '*'
            out.Variable_Name = variable_name
            out.Reporting_Frequency = 'hourly'
        if variable_name.startswith('Zone'):
            for j in range(len(lis_zone)):
                out = idf.newidfobject('OUTPUT:VARIABLE')
                out.Key_Value = lis_zone[j]
                out.Variable_Name = variable_name
                out.Reporting_Frequency = 'hourly'
        if variable_name.startswith('Zone Thermal Comfort'): # Attention, cette condition doit être après la précédente, car elle est plus spécifique
            for j in range(len(lis_zone)):
                out = idf.newidfobject('OUTPUT:VARIABLE')
                out.Key_Value = 'PEOPLE ' + lis_zone[j]
                out.Variable_Name = variable_name
                out.Reporting_Frequency = 'hourly'
        if variable_name.startswith('Surface'):
            out = idf.newidfobject('OUTPUT:VARIABLE')
            out.Key_Value = 'RDC:RUE_WALL_4_0_0'
            out.Variable_Name = variable_name
            out.Reporting_Frequency = 'hourly'

            out = idf.newidfobject('OUTPUT:VARIABLE')
            out.Key_Value = 'RDC:COUR_WALL_2_0_0'
            out.Variable_Name = variable_name
            out.Reporting_Frequency = 'hourly'

            out = idf.newidfobject('OUTPUT:VARIABLE')
            out.Key_Value = 'ETAGE:RUE_WALL_4_0_0'
            out.Variable_Name = variable_name
            out.Reporting_Frequency = 'hourly'

            out = idf.newidfobject('OUTPUT:VARIABLE')
            out.Key_Value = 'ETAGE:COUR_WALL_2_0_0'
            out.Variable_Name = variable_name
            out.Reporting_Frequency = 'hourly'

            # Pour le moment, on garde les 3 morceaux de toit, que l'on changera quand le toit aura été corrigé (il y aura une partie qui deviendra adiabatique)
            out = idf.newidfobject('OUTPUT:VARIABLE')
            out.Key_Value = 'COMBLES:RUE_ROOF_4_0_0'
            out.Variable_Name = variable_name
            out.Reporting_Frequency = 'hourly'

            out = idf.newidfobject('OUTPUT:VARIABLE')
            out.Key_Value = 'COMBLES:RUE_ROOF_0_0_0'
            out.Variable_Name = variable_name
            out.Reporting_Frequency = 'hourly'

            out = idf.newidfobject('OUTPUT:VARIABLE')
            out.Key_Value = 'COMBLES:COUR_ROOF_0_0_0'
            out.Variable_Name = variable_name
            out.Reporting_Frequency = 'hourly'
    
    # Ajout de la ligne nécessaire pour le confort
    for people in idf.idfobjects['PEOPLE']:
        people.Thermal_Comfort_Model_1_Type = 'Pierce'
        people.Thermal_Comfort_Model_2_Type = 'Fanger'
        people.Thermal_Comfort_Model_3_Type = 'AdaptiveASH55'
        people.Thermal_Comfort_Model_4_Type = 'AdaptiveCEN15251'
    
    return idf

In [38]:
def add_insulated_roof(idf):
    """ Cette fonction ajoute à un fichier idf déjà ouvert les éléments pour changer la composition du toit
    Elle ressort le fichier idf modifié"""
    
    # Création du matériau isolant du toit
    #for mat in idf.idfobjects['MATERIAL']:
    #    if 'Glass' in mat.Name:
    #        print(mat)
    mat = idf.newidfobject('MATERIAL')
    mat.Name = 'Glass fibre/wool - wool at 10C degrees_.20'
    mat.Roughness = 'Rough'
    mat.Thickness = 0.2
    mat.Conductivity = 0.037
    mat.Density = 16
    mat.Specific_Heat = 840
    mat.Thermal_Absorptance = 0.9
    mat.Solar_Absorptance = 0.6
    mat.Visible_Absorptance = 0.6

    # Création des deux nouvelles compositions du toit
    air_gap = []
    for cons in idf.idfobjects['CONSTRUCTION']:
        if 'Pitched roof' in cons.Name:
            air_gap.append(cons.Layer_2)
    cons = idf.newidfobject('CONSTRUCTION')
    cons.Name = 'Toit_isole'
    cons.Outside_Layer = 'Clay Tile (roofing)_.025'
    cons.Layer_2 = air_gap[0]
    cons.Layer_3 = 'Roofing Felt_.005'
    cons.Layer_4 = 'Glass fibre/wool - wool at 10C degrees_.20'
    cons.Layer_5 = 'Gypsum Plasterboard_.013'

    cons = idf.newidfobject('CONSTRUCTION')
    cons.Name = 'Toit_isole_Rev'
    cons.Layer_5 = 'Clay Tile (roofing)_.025'
    cons.Layer_4 = air_gap[0]
    cons.Layer_3 = 'Roofing Felt_.005'
    cons.Layer_2 = 'Glass fibre/wool - wool at 10C degrees_.20'
    cons.Outside_Layer = 'Gypsum Plasterboard_.013'

    # Mise en place de la nouvelle construction sur les éléments de toit
    for surf in idf.idfobjects['BUILDINGSURFACE:DETAILED']:
        if surf.Zone_Name.startswith('Combles') or surf.Zone_Name.startswith('Toit'):
            if 'roof' in surf.Construction_Name:
                if surf.Construction_Name == 'Pitched roof - Uninsulated - Medium weight':
                    surf.Construction_Name = 'Toit_isole'
                if surf.Construction_Name == 'Pitched roof - Uninsulated - Medium weight_Rev':
                    surf.Construction_Name == 'Toit_isole_Rev'
    
    return idf


In [39]:
def add_better_windows(idf):
    """Cette fonction prend en entrée un fichier idf et retourne le fichier idf équivalent mais avec de meilleurs vitrages"""

    # Création du matériau vitrage
    vitrage = idf.newidfobject('WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM')
    vitrage.Name = 'Simple Double_vitrage_eclaz_lumi - 1001'
    vitrage.UFactor = 1.1
    vitrage.Solar_Heat_Gain_Coefficient = 0.71
    vitrage.Visible_Transmittance = 0.83

    # Création de la construction vitre uniquement et de la création vitre + protections solaires
    cons1 = idf.newidfobject('CONSTRUCTION')
    cons1.Name = 'Double_vitrage_eclaz_lumi - 1001'
    cons1.Outside_Layer = 'Simple Double_vitrage_eclaz_lumi - 1001'

    cons2 = idf.newidfobject('CONSTRUCTION')
    cons2.Name = 'Double_vitrage_eclaz_lumi - 2001'
    cons2.Outside_Layer = '20001'
    cons2.Layer_2 = 'Simple Double_vitrage_eclaz_lumi - 1001'

    # Modification de la construction vitre + protections solaires pour le contrôle des protections solaires
    for control in idf.idfobjects['WINDOWSHADINGCONTROL']:
        control.Construction_with_Shading_Name = 'Double_vitrage_eclaz_lumi - 2001'

    # Modification de la construction vitre uniquement pour le contrôle des protections solaires
    for win in idf.idfobjects['FenestrationSurface:Detailed']:
        if win.Surface_Type == 'Window':
            win.Construction_Name = 'Double_vitrage_eclaz_lumi - 1001'
    
    return idf
    


In [40]:
def add_insulated_floor(idf):
    """ Cette fonction prend en entrée un fichier idf et retourne le fichier idf équivalent mais avec un sol isolé"""
    
    # Différentiation du cas avec et sans cave --> isolation par le dessus ou par le dessous
    cave = []
    for zone in idf.idfobjects['ZONE']:
        if zone.Name.startswith('Cave'):
            cave.append(zone.Name)
    if len(cave) >= 1:
        name_cons = 'Groundfloor_insulated_bottom'
    if len(cave) == 0:
        name_cons = 'Groundfloor_insulated_top'

    # Création du matériau isolant
    mat = idf.newidfobject('MATERIAL')
    mat.Name = 'Glass fibre/wool - wool at 10C degrees_.20'
    mat.Roughness = 'Rough'
    mat.Thickness = 0.2
    mat.Conductivity = 0.037
    mat.Density = 16
    mat.Specific_Heat = 840
    mat.Thermal_Absorptance = 0.9
    mat.Solar_Absorptance = 0.6
    mat.Visible_Absorptance = 0.6

    # Création des nouvelles constructions, pour le sol isolé par au-dessus et par en-dessous
    for cons in idf.idfobjects['CONSTRUCTION']:
        if cons.Name == 'Combined ground floor - Uninsulated - Medium weight':
            cons1 = cons
        if cons.Name == 'Combined ground floor - Uninsulated - Medium weight_Rev':
            cons1_rev = cons

    cons = idf.copyidfobject(cons1)
    cons.Name = 'Groundfloor_insulated_top'
    cons.Layer_3 = 'Glass fibre/wool - wool at 10C degrees_.20'
    cons.Layer_4 = 'Timber Flooring_.03' 

    cons = idf.copyidfobject(cons1)
    cons.Name = 'Groundfloor_insulated_bottom'
    cons.Outside_Layer = 'Glass fibre/wool - wool at 10C degrees_.20'
    cons.Layer_2 = 'Cast Concrete_.1'
    cons.Layer_3 = 'Floor/Roof Screed_.07' 
    cons.Layer_4 = 'Timber Flooring_.03'

    cons = idf.copyidfobject(cons1_rev)
    cons.Name = 'Groundfloor_insulated_bottom_rev'
    cons.Layer_4 = 'Glass fibre/wool - wool at 10C degrees_.20'
    cons.Layer_3 = 'Cast Concrete_.1'
    cons.Layer_2 = 'Floor/Roof Screed_.07' 
    cons.Outside_Layer = 'Timber Flooring_.03'

    # Remplacement des constructions dans les surfaces
    for surf in idf.idfobjects['BUILDINGSURFACE:DETAILED']:
        if surf.Zone_Name.startswith('RDC') and 'Floor' in surf.Surface_Type:
            surf.Construction_Name == name_cons

    for surf in idf.idfobjects['BUILDINGSURFACE:DETAILED']:
        if surf.Zone_Name.startswith('Cave') and 'Ceiling' in surf.Surface_Type:
            surf.Construction_Name = 'Groundfloor_insulated_bottom_rev'
    
    return(idf)

# Travail

In [41]:
idd_file = r'C:\EnergyPlusV9-4-0\Energy+.idd'
IDF.setiddname(idd_file)

### Sur un unique fichier

In [42]:
idf_file = str(Path(r'C:\Users\Tanchou\OneDrive - CSTBGroup\Documents\Simulations\v1\Validation_modelisation\base.idf'))
idf = IDF(idf_file)
idf = add_output_variables(idf)
#idf = add_insulated_roof(idf)
idf.saveas(Path(r'C:\Users\Tanchou\OneDrive - CSTBGroup\Documents\Simulations\v1\Validation_modelisation\base.idf'))


Changer windowmaterial:simpleglazingsystem, construction 1001 et 2001, window shading control, FenestrationSurface:Detailed

### Sur un grand nombre de fichiers

In [78]:
lis_cas_etude = ['base_eo_vnat_ps_sc1',
                 'base_eo_vnat_ps_sc2',
                 'base_ns_vnat_ps_sc1',
                 'base_ns_vnat_ps_sc2',
                 'cave_eo_vnat_ps_sc1',
                 'cave_eo_vnat_ps_sc2',
                 'cave_ns_vnat_ps_sc1',
                 'cave_ns_vnat_ps_sc2',
                 'tuffeau_eo_vnat_ps_sc1',
                 'tuffeau_eo_vnat_ps_sc2',
                 'tuffeau_ns_vnat_ps_sc1',
                 'tuffeau_ns_vnat_ps_sc2',
                 'tuffeau_cave_eo_vnat_ps_sc1',
                 'tuffeau_cave_eo_vnat_ps_sc2',
                 'tuffeau_cave_ns_vnat_ps_sc1',
                 'tuffeau_cave_ns_vnat_ps_sc2']

niveau = 2

for cas_etude in lis_cas_etude:
    idf_file = str(Path(r'C:\Users\Tanchou\OneDrive - CSTBGroup\Documents\Simulations\v1\Fichiers_IDF\Maison_centre_bourg') / ('Niveau_'+str(niveau)) / (cas_etude+'.idf'))
    idf = IDF(idf_file)
    idf = add_output_variables(idf)
    idf = add_insulated_roof(idf)
    idf = add_better_windows(idf)
    idf = add_insulated_floor(idf)

    if cas_etude.endswith('sc1'):
        idf.saveas(Path(r'C:\Users\Tanchou\OneDrive - CSTBGroup\Documents\Simulations\v1\Fichiers_IDF\Maison_centre_bourg') / ('Niveau_'+str(niveau)) / (cas_etude[:-1]+'15.idf'))
    if cas_etude.endswith('sc2'):
        idf.saveas(Path(r'C:\Users\Tanchou\OneDrive - CSTBGroup\Documents\Simulations\v1\Fichiers_IDF\Maison_centre_bourg') / ('Niveau_'+str(niveau)) / (cas_etude[:-1]+'16.idf'))